In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
st.table(resultados[['Cantidad', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
%load_ext autoreload
%autoreload 2

In [3]:
#carga del archivo:
data_car = pd.read_excel("data_car_completa.xlsx")

In [4]:
data_car.head(3)

,Year,Manufacturer,Model,co2,co2TailpipeGpm,fuelCost08,fuelCostA08,fuelType,ghgScore,highway08,range,Price
0,2023,Audi,e-tron S (20 inch wheels),0,0,950,0,Electricity,10,75,208,93800.0
1,2023,Audi,Q4 e-tron,0,0,700,0,Electricity,10,94,265,55200.0
2,2023,Audi,Q4 e-tron Sportback,0,0,750,0,Electricity,10,89,242,63200.0


In [8]:
# Calcula los percentiles para determinar los límites
percentiles = data_car['Price'].quantile([0, 0.33, 0.66, 1])

# Asigna etiquetas a las categorías según los percentiles y crea la nueva columna 'Gama'
data_car['Gama'] = pd.cut(data_car['Price'], bins=percentiles, labels=['Gama Baja', 'Gama Media', 'Gama Alta'], include_lowest=True)


In [9]:
data_car

,Year,Manufacturer,Model,co2,co2TailpipeGpm,fuelCost08,fuelCostA08,fuelType,ghgScore,highway08,range,Price,Gama
0,2023,Audi,e-tron S (20 inch wheels),0,0,950,0,Electricity,10,75,208,93800.0,Gama Alta
1,2023,Audi,Q4 e-tron,0,0,700,0,Electricity,10,94,265,55200.0,Gama Alta
2,2023,Audi,Q4 e-tron Sportback,0,0,750,0,Electricity,10,89,242,63200.0,Gama Alta
3,2023,BMW,i4 eDrive40 Gran Coupe (19 inch Wheels),0,0,700,0,Electricity,10,98,282,62600.0,Gama Alta
4,2023,BMW,i4 M50 Gran Coupe (19 inch wheels),0,0,750,0,Electricity,10,98,271,62900.0,Gama Alta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2023,Volkswagen,ID.4 AWD Pro S,0,0,700,0,Electricity,10,93,255,51440.0,Gama Media
90,2023,Volkswagen,ID.4 Pro,0,0,650,0,Electricity,10,98,275,51440.0,Gama Media
91,2023,Volkswagen,Jetta,265,265,1550,0,Regular,7,42,0,31340.0,Gama Baja
92,2023,Volvo,C40 Recharge twin,0,0,800,0,Electricity,10,80,226,55330.0,Gama Alta


In [10]:
data_car['Gama'].value_counts()

Gama Alta     31
Gama Baja     30
Gama Media    30
Name: Gama, dtype: int64

In [132]:
# Especifica las columnas clave donde no quieres valores faltantes o ceros
columnas_clave = ['Price', 'range']

# Elimina filas con valores faltantes en las columnas clave
data_car_sin_nulos = data_car.dropna(subset=columnas_clave)

# Elimina filas con ceros en las columnas clave
data_car_sin_ceros = data_car_sin_nulos[(data_car_sin_nulos[columnas_clave] != 0).all(axis=1)]


In [134]:
data_car

,Year,Manufacturer,Model,co2,co2TailpipeGpm,fuelCost08,fuelCostA08,fuelType,ghgScore,highway08,range,Price,Gama
0,2023,Audi,e-tron S (20 inch wheels),0,0,950,0,Electricity,10,75,208,93800.0,Gama Alta
1,2023,Audi,Q4 e-tron,0,0,700,0,Electricity,10,94,265,55200.0,Gama Alta
2,2023,Audi,Q4 e-tron Sportback,0,0,750,0,Electricity,10,89,242,63200.0,Gama Alta
3,2023,BMW,i4 eDrive40 Gran Coupe (19 inch Wheels),0,0,700,0,Electricity,10,98,282,62600.0,Gama Alta
4,2023,BMW,i4 M50 Gran Coupe (19 inch wheels),0,0,750,0,Electricity,10,98,271,62900.0,Gama Alta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2023,Volkswagen,ID.4 AWD Pro S,0,0,700,0,Electricity,10,93,255,51440.0,Gama Media
90,2023,Volkswagen,ID.4 Pro,0,0,650,0,Electricity,10,98,275,51440.0,Gama Media
91,2023,Volkswagen,Jetta,265,265,1550,0,Regular,7,42,0,31340.0,Gama Baja
92,2023,Volvo,C40 Recharge twin,0,0,800,0,Electricity,10,80,226,55330.0,Gama Alta


In [135]:
data_car['Gama'].value_counts()

Gama Alta     31
Gama Baja     30
Gama Media    30
Name: Gama, dtype: int64

In [92]:
#funcion para seleccionar los vehiculos de ambas gamas mas economicos.

def mejores_vehiculos(data_car, tipo_combustible, gama):
    # Filtrar por tipo de combustible y categoría de gama
    data_filtrada = data_car[(data_car['fuelType'] == tipo_combustible) & (data_car['Gama'] == gama)]
    
    # Encontrar el vehículo con el menor precio en la categoría
    mejor_vehiculo = data_filtrada.nsmallest(1, 'Price')
    
    return mejor_vehiculo

# Ejemplo de uso para encontrar los mejores vehículos eléctricos y de gasolina en gama baja
mejor_electrico_gama_baja = mejores_vehiculos(data_car, 'Electricity', 'Gama Baja')
mejor_gasolina_gama_baja = mejores_vehiculos(data_car, 'Regular', 'Gama Baja')

# Ejemplo de uso para encontrar los mejores vehículos eléctricos y de gasolina en gama media
mejor_electrico_gama_media = mejores_vehiculos(data_car, 'Electricity', 'Gama Media')
mejor_gasolina_gama_media = mejores_vehiculos(data_car, 'Regular', 'Gama Media')

# Mostrar los resultados
print("Mejor vehículo eléctrico en gama baja:")
print(mejor_electrico_gama_baja[['Manufacturer', 'Model', 'Gama', 'Price']])
print("\nMejor vehículo a gasolina en gama baja:")
print(mejor_gasolina_gama_baja[['Manufacturer', 'Model', 'Gama', 'Price']])
print("\nMejor vehículo eléctrico en gama media:")
print(mejor_electrico_gama_media[['Manufacturer', 'Model', 'Gama', 'Price']])
print("\nMejor vehículo a gasolina en gama media:")
print(mejor_gasolina_gama_media[['Manufacturer', 'Model', 'Gama', 'Price']])


Mejor vehículo eléctrico en gama baja:
   Manufacturer Model       Gama    Price
56       Nissan  LEAF  Gama Baja  31967.0

Mejor vehículo a gasolina en gama baja:
   Manufacturer Model       Gama    Price
46          Kia   Rio  Gama Baja  17490.0

Mejor vehículo eléctrico en gama media:
  Manufacturer    Model        Gama    Price
7    Chevrolet  Bolt EV  Gama Media  37500.0

Mejor vehículo a gasolina en gama media:
   Manufacturer    Model        Gama    Price
48        Lexus  UX 250h  Gama Media  37230.0


In [93]:
#dandole prioridad a variables como precio, menor co2 y mayor range vamos a seleccionar los mejores vehiculos


def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible):
    # Filtrar por gama y tipo de combustible
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

    # Verificar si hay vehículos que cumplen con los criterios
    if vehiculos_filtrados.empty:
        return "No se encontraron vehículos que cumplan con los criterios."

    # Priorizar el precio (mientras menos, mejor)
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

    # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])

    # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    return mejor_vehiculo[['Manufacturer', 'Model', 'Gama', 'Price', 'co2', 'range']]


In [94]:
seleccionar_mejor_vehiculo(data_car, 'Gama Media', 'Regular')


Manufacturer         Lexus
Model              UX 250h
Gama            Gama Media
Price              37230.0
co2                    209
range                    0
Name: 48, dtype: object

In [95]:
#que nos devuelva directamente solo dandole la data, los mejores vehiculos electricos y a gasolina de ambas gamas

def seleccionar_mejores_vehiculos(data_car):
    # Definir las combinaciones de gama y tipo de combustible
    combinaciones = [('Gama Baja', 'Electricity'),
                     ('Gama Media', 'Electricity'),
                     ('Gama Baja', 'Regular'),
                     ('Gama Media', 'Regular')]

    for gama, tipo_combustible in combinaciones:
        # Filtrar por gama y tipo de combustible
        vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

        # Verificar si hay vehículos que cumplen con los criterios
        if not vehiculos_filtrados.empty:
            # Priorizar el precio (mientras menos, mejor)
            vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

            # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
            vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])

            # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
            mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

            print(f"Mejor Vehículo {tipo_combustible} en {gama}:")
            print(mejor_vehiculo[['Manufacturer', 'Model', 'Gama', 'Price', 'co2', 'range']])
            print("\n")
        else:
            print(f"No se encontraron vehículos para {tipo_combustible} en {gama}.\n")





In [96]:
seleccionar_mejores_vehiculos(data_car)

Mejor Vehículo Electricity en Gama Baja:
Manufacturer       Nissan
Model                LEAF
Gama            Gama Baja
Price             31967.0
co2                     0
range                 149
Name: 56, dtype: object


Mejor Vehículo Electricity en Gama Media:
Manufacturer        Hyundai
Model           Ioniq 6 RWD
Gama             Gama Media
Price               45500.0
co2                       0
range                   361
Name: 28, dtype: object


Mejor Vehículo Regular en Gama Baja:
Manufacturer          Kia
Model                Niro
Gama            Gama Baja
Price             23990.0
co2                    42
range                 385
Name: 42, dtype: object


Mejor Vehículo Regular en Gama Media:
Manufacturer         Lexus
Model              UX 250h
Gama            Gama Media
Price              37230.0
co2                    209
range                    0
Name: 48, dtype: object




In [137]:
def seleccionar_mejores_vehiculos(data_car, inversion_inicial):
    # Calcular montos a gastar en vehículos eléctricos y a gasolina
    monto_electricos = inversion_inicial * 0.4
    monto_gasolina = inversion_inicial * 0.6

    # Seleccionar el mejor vehículo eléctrico
    resultados_electricos = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Electricity', monto_electricos)

    # Seleccionar el mejor vehículo a gasolina
    resultados_gasolina = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Regular', monto_gasolina)

    return resultados_electricos, resultados_gasolina

def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible, inversion_a_gastar):
    # Filtrar por gama y tipo de combustible
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

    # Verificar si hay vehículos que cumplen con los criterios
    if vehiculos_filtrados.empty:
        return "No se encontraron vehículos que cumplan con los criterios."

    # Priorizar el precio (mientras menos, mejor)
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

    # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])


    # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    # Calcular cuántos vehículos se pueden comprar con el monto a gastar
    cantidad_vehiculos = int(inversion_a_gastar / mejor_vehiculo['Price'])
   

    # Crear una lista con la información del vehículo sugerido
    vehiculos_sugeridos = [mejor_vehiculo] * cantidad_vehiculos

    return pd.DataFrame(vehiculos_sugeridos)

# Ejemplo de uso
resultados_electricos, resultados_gasolina = seleccionar_mejores_vehiculos(data_car, 1000000)

# Imprimir resultados como DataFrames con las columnas seleccionadas
print("Vehículos Eléctricos Gama Baja:\n")
print(resultados_electricos[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

print("\nVehículos Gasolina Gama Baja:\n")
print(resultados_gasolina[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])


Vehículos Eléctricos Gama Baja:

    Year Manufacturer Model     fuelType    Price       Gama
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja

Vehículos Gasolina Gama Baja:

    Year Manufacturer Model fuelType    Price       Gama
42  2023          Kia  Niro  Regular  23990.0  Gama Baja
42  2023          Kia  Niro 

In [138]:
def seleccionar_mejores_vehiculos(data_car, inversion_inicial, porcentaje_electrico):
    # Convertir el porcentaje a decimal dividiendo por 100
    porcentaje_decimal = porcentaje_electrico / 100

    # Calcular montos a gastar en vehículos eléctricos y a gasolina
    monto_electricos = inversion_inicial * porcentaje_decimal
    monto_gasolina = inversion_inicial * (1 - porcentaje_decimal)

    # Seleccionar el mejor vehículo eléctrico
    resultados_electricos = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Electricity', monto_electricos)

    # Seleccionar el mejor vehículo a gasolina
    resultados_gasolina = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Regular', monto_gasolina)

    # Imprimir resultados de vehículos eléctricos
    print("Vehículos Eléctricos Gama Baja:\n")
    print(resultados_electricos[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

    # Imprimir resultados de vehículos a gasolina
    print("\nVehículos Gasolina Gama Baja:\n")
    print(resultados_gasolina[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible, inversion_a_gastar):
    # Filtrar por gama y tipo de combustible
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

    # Verificar si hay vehículos que cumplen con los criterios
    if vehiculos_filtrados.empty:
        print("No se encontraron vehículos que cumplan con los criterios.")
        return pd.DataFrame()
    # Priorizar el precio (mientras menos, mejor)
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

    # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])


    # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    # Calcular cuántos vehículos se pueden comprar con el monto a gastar
    cantidad_vehiculos = int(inversion_a_gastar / mejor_vehiculo['Price'])
   

    # Crear una lista con la información del vehículo sugerido
    vehiculos_sugeridos = [mejor_vehiculo] * cantidad_vehiculos

    return pd.DataFrame(vehiculos_sugeridos)

   




In [139]:
# Ejemplo de uso con porcentaje eléctrico como 40 (se convertirá a 0.4)
seleccionar_mejores_vehiculos(data_car, 1000000, 50)


Vehículos Eléctricos Gama Baja:

    Year Manufacturer Model     fuelType    Price       Gama
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  G

# MODELO DEFINITIVO

In [140]:
def seleccionar_mejores_vehiculos(data_car, inversion_inicial, porcentaje_electrico, porcentaje_gama_media):
    # Convertir los porcentajes a decimales dividiendo por 100
    porcentaje_electrico_decimal = porcentaje_electrico / 100
    porcentaje_gama_media_decimal = porcentaje_gama_media / 100

    # Calcular montos a gastar en vehículos eléctricos y a gasolina para gama media
    monto_gama_media = inversion_inicial * porcentaje_gama_media_decimal
    monto_electricos_gama_media = monto_gama_media * porcentaje_electrico_decimal
    monto_gasolina_gama_media = monto_gama_media * (1 - porcentaje_electrico_decimal)

    # Calcular montos a gastar en vehículos eléctricos y a gasolina para gama baja
    monto_gama_baja = inversion_inicial * (1 - porcentaje_gama_media_decimal)
    monto_electricos_gama_baja = monto_gama_baja * porcentaje_electrico_decimal
    monto_gasolina_gama_baja = monto_gama_baja * (1 - porcentaje_electrico_decimal)

    # Seleccionar el mejor vehículo eléctrico para gama media
    resultados_electricos_gama_media = seleccionar_mejor_vehiculo(data_car, 'Gama Media', 'Electricity', monto_electricos_gama_media)

    # Seleccionar el mejor vehículo a gasolina para gama media
    resultados_gasolina_gama_media = seleccionar_mejor_vehiculo(data_car, 'Gama Media', 'Regular', monto_gasolina_gama_media)

    # Seleccionar el mejor vehículo eléctrico para gama baja
    resultados_electricos_gama_baja = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Electricity', monto_electricos_gama_baja)

    # Seleccionar el mejor vehículo a gasolina para gama baja
    resultados_gasolina_gama_baja = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Regular', monto_gasolina_gama_baja)

    # Imprimir resultados de vehículos eléctricos y a gasolina para gama media
    print("Vehículos Eléctricos Gama Media:\n")
    print(resultados_electricos_gama_media[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

    print("\nVehículos Gasolina Gama Media:\n")
    print(resultados_gasolina_gama_media[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

    # Imprimir resultados de vehículos eléctricos y a gasolina para gama baja
    print("\nVehículos Eléctricos Gama Baja:\n")
    print(resultados_electricos_gama_baja[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])

    print("\nVehículos Gasolina Gama Baja:\n")
    print(resultados_gasolina_gama_baja[['Year', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])



In [141]:
def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible, inversion_a_gastar):
    # Filtrar por gama y tipo de combustible
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

    # Verificar si hay vehículos que cumplen con los criterios
    if vehiculos_filtrados.empty:
        print(f"No se encontraron vehículos que cumplan con los criterios para Gama {gama}.")
        return pd.DataFrame()
    # Priorizar el precio (mientras menos, mejor)
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

    # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])


    # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    # Calcular cuántos vehículos se pueden comprar con el monto a gastar
    cantidad_vehiculos = int(inversion_a_gastar / mejor_vehiculo['Price'])
   

    # Crear una lista con la información del vehículo sugerido
    vehiculos_sugeridos = [mejor_vehiculo] * cantidad_vehiculos

    return pd.DataFrame(vehiculos_sugeridos)


In [142]:
seleccionar_mejores_vehiculos(data_car, 1000000, 30, 20)

Vehículos Eléctricos Gama Media:

    Year Manufacturer        Model     fuelType    Price        Gama
28  2023      Hyundai  Ioniq 6 RWD  Electricity  45500.0  Gama Media

Vehículos Gasolina Gama Media:

    Year Manufacturer    Model fuelType    Price        Gama
48  2023        Lexus  UX 250h  Regular  37230.0  Gama Media
48  2023        Lexus  UX 250h  Regular  37230.0  Gama Media
48  2023        Lexus  UX 250h  Regular  37230.0  Gama Media

Vehículos Eléctricos Gama Baja:

    Year Manufacturer Model     fuelType    Price       Gama
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja
56  2023       Nissan  LEAF  Electricity  31967.0  Gama Baja

Vehículos Gasolina Gama Baja

# Modelo Ordenado para Streamlit

In [143]:
def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible, inversion_a_gastar):
    # Filtrar por gama y tipo de combustible
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]

    # Verificar si hay vehículos que cumplen con los criterios
    if vehiculos_filtrados.empty:
        print(f"No se encontraron vehículos que cumplan con los criterios para Gama {gama}.")
        return pd.DataFrame()
    # Priorizar el precio (mientras menos, mejor)
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')

    # Luego, dentro de los más económicos, priorizar las bajas emisiones de CO2 y mayor rango
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])


    # Seleccionar el vehículo con el precio más bajo, bajas emisiones de CO2 y mayor rango
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    # Calcular cuántos vehículos se pueden comprar con el monto a gastar
    cantidad_vehiculos = int(inversion_a_gastar / mejor_vehiculo['Price'])
   

    # Crear una lista con la información del vehículo sugerido
    vehiculos_sugeridos = [mejor_vehiculo] * cantidad_vehiculos

    return pd.DataFrame(vehiculos_sugeridos)


In [12]:
def seleccionar_mejores_vehiculos(data_car, inversion_inicial, porcentaje_electrico, porcentaje_gama_media):
    # Convertir los porcentajes a decimales dividiendo por 100
    porcentaje_electrico_decimal = porcentaje_electrico / 100
    porcentaje_gama_media_decimal = porcentaje_gama_media / 100

    # Calcular montos a gastar en vehículos eléctricos y a gasolina para gama media
    monto_gama_media = inversion_inicial * porcentaje_gama_media_decimal
    monto_electricos_gama_media = monto_gama_media * porcentaje_electrico_decimal
    monto_gasolina_gama_media = monto_gama_media * (1 - porcentaje_electrico_decimal)

    # Calcular montos a gastar en vehículos eléctricos y a gasolina para gama baja
    monto_gama_baja = inversion_inicial * (1 - porcentaje_gama_media_decimal)
    monto_electricos_gama_baja = monto_gama_baja * porcentaje_electrico_decimal
    monto_gasolina_gama_baja = monto_gama_baja * (1 - porcentaje_electrico_decimal)

    # Seleccionar el mejor vehículo eléctrico para gama media
    resultados_electricos_gama_media = seleccionar_mejor_vehiculo(data_car, 'Gama Media', 'Electricity', monto_electricos_gama_media)

    # Seleccionar el mejor vehículo a gasolina para gama media
    resultados_gasolina_gama_media = seleccionar_mejor_vehiculo(data_car, 'Gama Media', 'Regular', monto_gasolina_gama_media)

    # Seleccionar el mejor vehículo eléctrico para gama baja
    resultados_electricos_gama_baja = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Electricity', monto_electricos_gama_baja)

    # Seleccionar el mejor vehículo a gasolina para gama baja
    resultados_gasolina_gama_baja = seleccionar_mejor_vehiculo(data_car, 'Gama Baja', 'Regular', monto_gasolina_gama_baja)

    # Concatenar todos los resultados en un solo DataFrame
    todos_los_resultados = pd.concat([
        resultados_electricos_gama_media,
        resultados_gasolina_gama_media,
        resultados_electricos_gama_baja,
        resultados_gasolina_gama_baja
    ])

    # Agrupar por Manufacturer, Model y Gama y contar la cantidad
    resultados_agrupados = todos_los_resultados.groupby(['Manufacturer', 'Model', 'fuelType','Price', 'Gama']).size().reset_index(name='Cantidad')

    # Definir el orden personalizado para gamas y tipos de combustible
    orden_gamas = {'Gama Baja Electricidad': 1, 'Gama Baja Gasolina': 2, 'Gama Media Electricidad': 3, 'Gama Media Gasolina': 4}
    orden_combustible = {'Electricity': 1, 'Regular': 2}

    # Crear columnas adicionales para ordenar
    resultados_agrupados['Orden_Gama'] = resultados_agrupados['Gama'] + ' ' + resultados_agrupados['fuelType']
    resultados_agrupados['Orden_Gama'] = resultados_agrupados['Orden_Gama'].map(orden_gamas)
    resultados_agrupados['Orden_Combustible'] = resultados_agrupados['fuelType'].map(orden_combustible)

    # Ordenar el DataFrame
    resultados_agrupados = resultados_agrupados.sort_values(by=['Orden_Gama', 'Orden_Combustible', 'Manufacturer', 'Model'])

    # Eliminar las columnas adicionales creadas para ordenar
    resultados_agrupados = resultados_agrupados.drop(['Orden_Gama', 'Orden_Combustible'], axis=1)

    # Imprimir resultados de vehículos sugeridos y su cantidad
    print("\nVehículos Sugeridos y Cantidad:\n")
    print(resultados_agrupados[['Cantidad', 'Manufacturer', 'Model', 'fuelType', 'Price', 'Gama']])
 
def seleccionar_mejor_vehiculo(data_car, gama, tipo_combustible, inversion_a_gastar):
    # Filtrar los vehículos que cumplan con los criterios
    vehiculos_filtrados = data_car[(data_car['Gama'] == gama) & (data_car['fuelType'] == tipo_combustible)]
    if vehiculos_filtrados.empty:
        print(f"No se encontraron vehículos que cumplan con los criterios para Gama {gama}.")
        return pd.DataFrame()

    vehiculos_filtrados = vehiculos_filtrados.sort_values(by='Price')
    vehiculos_filtrados = vehiculos_filtrados.sort_values(by=['co2', 'range'], ascending=[True, False])
    mejor_vehiculo = vehiculos_filtrados.iloc[0].copy()

    cantidad_vehiculos = int(inversion_a_gastar / mejor_vehiculo['Price'])
    vehiculos_sugeridos = [mejor_vehiculo] * cantidad_vehiculos

    return pd.DataFrame(vehiculos_sugeridos)




In [13]:
seleccionar_mejores_vehiculos(data_car, 2000000, 40, 30)


Vehículos Sugeridos y Cantidad:

   Cantidad Manufacturer        Model     fuelType    Price        Gama
0         5      Hyundai  Ioniq 6 RWD  Electricity  45500.0  Gama Media
3        17       Nissan         LEAF  Electricity  31967.0   Gama Baja
1        35          Kia         Niro      Regular  23990.0   Gama Baja
2         9        Lexus      UX 250h      Regular  37230.0  Gama Media
